<a href="https://colab.research.google.com/github/vgaurav3011/EIP-3.0-/blob/master/FinalAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls
%tensorflow_version 1.x
!pip install bokeh
import cv2
import json
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16,InceptionV3, ResNet50V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from keras.preprocessing.image import ImageDataGenerator
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T
import keras
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation
    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape
train_df.head()
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=128)
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=True)
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace resized/63.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
drive/  hvc_annotations.csv  resized/  sample_data/


{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [15]:
backbone = ResNet50V2(
    weights=None, 
    include_top=True, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output



def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

from keras.callbacks import *



# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
model.summary()
 # checkpoint
filepath="/content/drive/My Drive/Colab Notebooks/weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_loss', verbose=1,save_weights_only=False, save_best_only=True)  
 

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:
def find_lr(model, start_lr, end_lr):
    finder = LRFinder(start_lr, end_lr, len(train_gen))
    weights = model.get_weights()    
    try:
        history = model.fit_generator(
            generator=train_gen,
            validation_data=valid_gen,
            epochs=1,
            verbose=1,
            callbacks=[finder],
        )   
    finally:    
        model.set_weights(weights)    
    return finder
    
class LRFinder(Callback):    
    def __init__(self, start_lr=1e-5, end_lr=10, step_size=None, beta=.98):
        super().__init__()
        
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.step_size = step_size
        self.beta = beta
        self.lr_mult = (end_lr/start_lr)**(1/step_size)
        
    def on_train_begin(self, logs=None):
        self.best_loss = 1e9
        self.avg_loss = 0
        self.losses, self.smoothed_losses, self.lrs, self.iterations = [], [], [], []
        self.iteration = 0
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.start_lr)
        
    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        self.iteration += 1
        
        self.avg_loss = self.beta * self.avg_loss + (1 - self.beta) * loss
        smoothed_loss = self.avg_loss / (1 - self.beta**self.iteration)
        
        # Check if the loss is not exploding
        if self.iteration>1 and smoothed_loss > self.best_loss * 4:
            self.model.stop_training = True
            return

        if smoothed_loss < self.best_loss or self.iteration==1:
            self.best_loss = smoothed_loss
        
        lr = self.start_lr * (self.lr_mult**self.iteration)
        
        self.losses.append(loss)
        self.smoothed_losses.append(smoothed_loss)
        self.lrs.append(lr)
        self.iterations.append(self.iteration)        
        
        K.set_value(self.model.optimizer.lr, lr)  

    def plot(self, lskip=10, rskip=10):
        lrs = self.lrs[lskip:-rskip]
        losses = self.smoothed_losses[lskip:-rskip]

        output_notebook()
        p = figure(title='Learning Rate Finder', x_axis_label='LR', y_axis_label='Loss')
        p.line(lrs, losses)
        show(p)
        
        best_idxs = np.argpartition(losses, 15)[:15]
        best_lrs = np.take(lrs, best_idxs)
        print(f"Best LRs: {best_lrs}")    
#Cyclic Learning Rate
class CyclicLR(Callback):
  

    def __init__(self, base_lr=0.00000005, max_lr=0.009, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
#finder = find_lr(model, 1e-7, 1e-3)
#finder.plot()


In [20]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular = CyclicLR(1e-5,6e-3,mode='triangular')
history=model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 128,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
  2/312 [..............................] - ETA: 35:14 - loss: 9.4664 - gender_output_loss: 0.6932 - image_quality_output_loss: 1.0994 - age_output_loss: 1.6091 - weight_output_loss: 1.3859 - bag_output_loss: 1.0987 - footwear_output_loss: 1.0977 - pose_output_loss: 1.0976 - emotion_output_loss: 1.3848 - gender_output_acc: 0.4062 - image_quality_output_acc: 0.1406 - age_output_acc: 0.3281 - weight_output_acc: 0.2812 - bag_output_acc: 0.2500 - footwear_output_acc: 0.5156 - pose_output_acc: 0.6094 - emotion_output_acc: 0.7031  

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.484488). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


312/312 [==============================] - 173s 556ms/step - loss: 8.4368 - gender_output_loss: 0.6860 - image_quality_output_loss: 1.0190 - age_output_loss: 1.5011 - weight_output_loss: 1.1359 - bag_output_loss: 0.9800 - footwear_output_loss: 1.0562 - pose_output_loss: 0.9818 - emotion_output_loss: 1.0767 - gender_output_acc: 0.5673 - image_quality_output_acc: 0.5452 - age_output_acc: 0.4065 - weight_output_acc: 0.6185 - bag_output_acc: 0.5538 - footwear_output_acc: 0.4462 - pose_output_acc: 0.6187 - emotion_output_acc: 0.7159 - val_loss: 8.0088 - val_gender_output_loss: 0.6849 - val_image_quality_output_loss: 0.9928 - val_age_output_loss: 1.4529 - val_weight_output_loss: 0.9850 - val_bag_output_loss: 0.9444 - val_footwear_output_loss: 1.0468 - val_pose_output_loss: 0.9233 - val_emotion_output_loss: 0.9787 - val_gender_output_acc: 0.5641 - val_image_quality_output_acc: 0.5536 - val_age_output_acc: 0.3667 - val_weight_output_acc: 0.6443 - val_bag_output_acc: 0.5464 - val_footwear_outpu

In [21]:
plot_stats(history.history, clr_triangular.history)

NameError: ignored

In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 1e-5, 1.2e-3)
finder.plot()


Epoch 1/1
339/339 [==============================] - 107s 317ms/step - loss: 4.8641 - gender_output_loss: 0.0613 - image_quality_output_loss: 0.8824 - age_output_loss: 1.0624 - weight_output_loss: 0.7606 - bag_output_loss: 0.6962 - footwear_output_loss: 0.4879 - pose_output_loss: 0.1299 - emotion_output_loss: 0.7836 - gender_output_acc: 0.9811 - image_quality_output_acc: 0.5797 - age_output_acc: 0.5171 - weight_output_acc: 0.6898 - bag_output_acc: 0.7085 - footwear_output_acc: 0.7911 - pose_output_acc: 0.9580 - emotion_output_acc: 0.7159 - val_loss: 4.7975 - val_gender_output_loss: 0.0574 - val_image_quality_output_loss: 0.8780 - val_age_output_loss: 1.0444 - val_weight_output_loss: 0.7451 - val_bag_output_loss: 0.6804 - val_footwear_output_loss: 0.4751 - val_pose_output_loss: 0.1359 - val_emotion_output_loss: 0.7813 - val_gender_output_acc: 0.9803 - val_image_quality_output_acc: 0.5751 - val_age_output_acc: 0.5306 - val_weight_output_acc: 0.6923 - val_bag_output_acc: 0.7184 - val_foot

Best LRs: [0.00141808 0.00090128 0.00105374 0.00107034 0.00070187 0.0010872
 0.00110433 0.00112172 0.00119409 0.00069098 0.0012129  0.001232
 0.00125141 0.00127112 0.00117557]


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        #zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
    )
)
clr_triangular = CyclicLR(1e-5,1.2e-3,mode='triangular')
history=model.fit_generator(  
    steps_per_epoch=20000 // 64,  validation_steps=2000 // 128,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

Epoch 1/25
 91/312 [=======>......................] - ETA: 1:50 - loss: 5.8211 - gender_output_loss: 0.1582 - image_quality_output_loss: 0.9565 - age_output_loss: 1.2878 - weight_output_loss: 0.8853 - bag_output_loss: 0.7706 - footwear_output_loss: 0.6429 - pose_output_loss: 0.3090 - emotion_output_loss: 0.8109 - gender_output_acc: 0.9464 - image_quality_output_acc: 0.5474 - age_output_acc: 0.4234 - weight_output_acc: 0.6514 - bag_output_acc: 0.6624 - footwear_output_acc: 0.7325 - pose_output_acc: 0.8953 - emotion_output_acc: 0.7198

In [0]:
plot_stats(history.history, clr_triangular.history)

In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 1e-6, 1.2e-3)
finder.plot()


Epoch 1/1
339/339 [==============================] - 110s 324ms/step - loss: 3.0395 - gender_output_loss: 0.0203 - image_quality_output_loss: 0.5843 - age_output_loss: 0.5530 - weight_output_loss: 0.5383 - bag_output_loss: 0.3927 - footwear_output_loss: 0.2756 - pose_output_loss: 0.0265 - emotion_output_loss: 0.6489 - gender_output_acc: 0.9925 - image_quality_output_acc: 0.7407 - age_output_acc: 0.7715 - weight_output_acc: 0.7865 - bag_output_acc: 0.8485 - footwear_output_acc: 0.8825 - pose_output_acc: 0.9911 - emotion_output_acc: 0.7540 - val_loss: 2.9218 - val_gender_output_loss: 0.0083 - val_image_quality_output_loss: 0.5496 - val_age_output_loss: 0.5644 - val_weight_output_loss: 0.5113 - val_bag_output_loss: 0.3659 - val_footwear_output_loss: 0.2634 - val_pose_output_loss: 0.0227 - val_emotion_output_loss: 0.6362 - val_gender_output_acc: 0.9976 - val_image_quality_output_acc: 0.7646 - val_age_output_acc: 0.7865 - val_weight_output_acc: 0.8069 - val_bag_output_acc: 0.8603 - val_foot

Best LRs: [2.96119593e-05 1.79255303e-05 1.58115057e-05 1.61456807e-05
 1.64869184e-05 1.68353682e-05 1.71911825e-05 1.75545169e-05
 2.89419888e-04 1.83043850e-05 1.86912468e-05 1.90862849e-05
 3.14674023e-04 2.95536756e-04 1.94896722e-05]


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        rotation_range=45,
    )
)
clr_triangular = CyclicLR(1e-6,6e-4,mode='exp_range')
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

Epoch 1/25


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


  1/625 [..............................] - ETA: 26:15 - loss: 3.3865 - gender_output_loss: 0.0059 - image_quality_output_loss: 0.9324 - age_output_loss: 0.5533 - weight_output_loss: 0.5244 - bag_output_loss: 0.3944 - footwear_output_loss: 0.3019 - pose_output_loss: 0.0414 - emotion_output_loss: 0.6327 - gender_output_acc: 1.0000 - image_quality_output_acc: 0.6250 - age_output_acc: 0.7500 - weight_output_acc: 0.7500 - bag_output_acc: 0.8750 - footwear_output_acc: 0.8750 - pose_output_acc: 0.9688 - emotion_output_acc: 0.7188

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (4.010322). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


  2/625 [..............................] - ETA: 35:06 - loss: 3.0116 - gender_output_loss: 0.0079 - image_quality_output_loss: 0.7008 - age_output_loss: 0.4400 - weight_output_loss: 0.5399 - bag_output_loss: 0.4058 - footwear_output_loss: 0.3096 - pose_output_loss: 0.0394 - emotion_output_loss: 0.5683 - gender_output_acc: 1.0000 - image_quality_output_acc: 0.7188 - age_output_acc: 0.8594 - weight_output_acc: 0.7656 - bag_output_acc: 0.8750 - footwear_output_acc: 0.8594 - pose_output_acc: 0.9688 - emotion_output_acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (2.009649). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 149s 239ms/step - loss: 3.0200 - gender_output_loss: 0.0184 - image_quality_output_loss: 0.5792 - age_output_loss: 0.5485 - weight_output_loss: 0.5349 - bag_output_loss: 0.3916 - footwear_output_loss: 0.2737 - pose_output_loss: 0.0259 - emotion_output_loss: 0.6477 - gender_output_acc: 0.9931 - image_quality_output_acc: 0.7416 - age_output_acc: 0.7751 - weight_output_acc: 0.7911 - bag_output_acc: 0.8480 - footwear_output_acc: 0.8828 - pose_output_acc: 0.9919 - emotion_output_acc: 0.7579 - val_loss: 2.6972 - val_gender_output_loss: 0.0024 - val_image_quality_output_loss: 0.5282 - val_age_output_loss: 0.4658 - val_weight_output_loss: 0.4827 - val_bag_output_loss: 0.3352 - val_footwear_output_loss: 0.2477 - val_pose_output_loss: 0.0117 - val_emotion_output_loss: 0.6236 - val_gender_output_acc: 0.9995 - val_image_quality_output_acc: 0.7692 - val_age_output_acc: 0.8175 - val_weight_output_acc: 0.8261 - val_bag_output_acc: 0.8745 - val_footwear_outpu

In [0]:
plot_stats(history.history, clr_triangular.history)

In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 1e-6, 1.2e-3)
finder.plot()


Epoch 1/1
339/339 [==============================] - 109s 321ms/step - loss: 1.5320 - gender_output_loss: 0.0163 - image_quality_output_loss: 0.1488 - age_output_loss: 0.2564 - weight_output_loss: 0.3373 - bag_output_loss: 0.1636 - footwear_output_loss: 0.1015 - pose_output_loss: 0.0200 - emotion_output_loss: 0.4881 - gender_output_acc: 0.9932 - image_quality_output_acc: 0.9462 - age_output_acc: 0.9111 - weight_output_acc: 0.8803 - bag_output_acc: 0.9408 - footwear_output_acc: 0.9619 - pose_output_acc: 0.9936 - emotion_output_acc: 0.8219 - val_loss: 1.5775 - val_gender_output_loss: 0.0081 - val_image_quality_output_loss: 0.1576 - val_age_output_loss: 0.3201 - val_weight_output_loss: 0.3400 - val_bag_output_loss: 0.1958 - val_footwear_output_loss: 0.0873 - val_pose_output_loss: 0.0104 - val_emotion_output_loss: 0.4583 - val_gender_output_acc: 0.9971 - val_image_quality_output_acc: 0.9464 - val_age_output_acc: 0.8803 - val_weight_output_acc: 0.8740 - val_bag_output_acc: 0.9210 - val_foot

Best LRs: [2.39761914e-04 3.56746463e-04 2.60683005e-04 2.55287522e-04
 2.50003712e-04 2.44829264e-04 9.77375001e-06 9.57145796e-06
 9.37335284e-06 3.49362708e-04 3.14674023e-04 9.17934801e-06
 2.29939687e-04 2.71818480e-04 3.35050511e-04]


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=True,
        vertical_flip=False,
        zoom_range=[0.4,1.0],
    )
)
clr_triangular = CyclicLR(5e-6,2.5e-4,mode='triangular2')
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
  1/625 [..............................] - ETA: 27:31 - loss: 1.7208 - gender_output_loss: 0.0033 - image_quality_output_loss: 0.1586 - age_output_loss: 0.2236 - weight_output_loss: 0.3366 - bag_output_loss: 0.0491 - footwear_output_loss: 0.0473 - pose_output_loss: 0.0201 - emotion_output_loss: 0.8821 - gender_output_acc: 1.0000 - image_quality_output_acc: 0.9062 - age_output_acc: 0.9375 - weight_output_acc: 0.8438 - bag_output_acc: 0.9688 - footwear_output_acc: 0.9688 - pose_output_acc: 1.0000 - emotion_output_acc: 0.7188

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (4.253723). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


  2/625 [..............................] - ETA: 36:59 - loss: 1.5044 - gender_output_loss: 0.0028 - image_quality_output_loss: 0.1343 - age_output_loss: 0.1847 - weight_output_loss: 0.3197 - bag_output_loss: 0.1131 - footwear_output_loss: 0.0428 - pose_output_loss: 0.0104 - emotion_output_loss: 0.6965 - gender_output_acc: 1.0000 - image_quality_output_acc: 0.9531 - age_output_acc: 0.9531 - weight_output_acc: 0.8750 - bag_output_acc: 0.9375 - footwear_output_acc: 0.9844 - pose_output_acc: 1.0000 - emotion_output_acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (2.132334). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 150s 240ms/step - loss: 1.4967 - gender_output_loss: 0.0131 - image_quality_output_loss: 0.1477 - age_output_loss: 0.2407 - weight_output_loss: 0.3362 - bag_output_loss: 0.1577 - footwear_output_loss: 0.0980 - pose_output_loss: 0.0202 - emotion_output_loss: 0.4830 - gender_output_acc: 0.9949 - image_quality_output_acc: 0.9492 - age_output_acc: 0.9198 - weight_output_acc: 0.8810 - bag_output_acc: 0.9406 - footwear_output_acc: 0.9660 - pose_output_acc: 0.9935 - emotion_output_acc: 0.8249 - val_loss: 1.1034 - val_gender_output_loss: 0.0025 - val_image_quality_output_loss: 0.0775 - val_age_output_loss: 0.1356 - val_weight_output_loss: 0.2778 - val_bag_output_loss: 0.0914 - val_footwear_output_loss: 0.0562 - val_pose_output_loss: 0.0030 - val_emotion_output_loss: 0.4595 - val_gender_output_acc: 0.9990 - val_image_quality_output_acc: 0.9814 - val_age_output_acc: 0.9768 - val_weight_output_acc: 0.9183 - val_bag_output_acc: 0.9763 - val_footwear_outpu

In [0]:
plot_stats(history.history, clr_triangular.history)

In [0]:
opt = RMSprop()
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
      
    metrics=["accuracy"]
)
finder = find_lr(model, 1e-6, 2.5e-4)
finder.plot()

Epoch 1/1
339/339 [==============================] - 109s 322ms/step - loss: 1.1458 - gender_output_loss: 0.0110 - image_quality_output_loss: 0.0839 - age_output_loss: 0.1846 - weight_output_loss: 0.2598 - bag_output_loss: 0.1109 - footwear_output_loss: 0.0640 - pose_output_loss: 0.0178 - emotion_output_loss: 0.4138 - gender_output_acc: 0.9960 - image_quality_output_acc: 0.9750 - age_output_acc: 0.9399 - weight_output_acc: 0.9132 - bag_output_acc: 0.9599 - footwear_output_acc: 0.9783 - pose_output_acc: 0.9935 - emotion_output_acc: 0.8444 - val_loss: 0.7627 - val_gender_output_loss: 0.0014 - val_image_quality_output_loss: 0.0337 - val_age_output_loss: 0.0902 - val_weight_output_loss: 0.1956 - val_bag_output_loss: 0.0520 - val_footwear_output_loss: 0.0287 - val_pose_output_loss: 0.0025 - val_emotion_output_loss: 0.3586 - val_gender_output_acc: 1.0000 - val_image_quality_output_acc: 0.9960 - val_age_output_acc: 0.9874 - val_weight_output_acc: 0.9462 - val_bag_output_acc: 0.9900 - val_foot

Best LRs: [3.10830146e-05 3.56236495e-06 3.00867996e-05 1.79741556e-06
 1.76837728e-06 3.50481291e-06 1.73980814e-06 1.71170054e-06
 3.31754949e-05 3.26395258e-05 1.68404704e-06 3.21122156e-05
 3.15934244e-05 3.62086204e-06 3.05808507e-05]


In [0]:
	# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    augmentation=ImageDataGenerator(
        zca_whitening=True,
        horizontal_flip=False,
        vertical_flip=True,
        
    )
)
clr_triangular = CyclicLR(6e-6,3e-5,mode='triangular2')
history=model.fit_generator(  
    steps_per_epoch=20000 // 32,  validation_steps=2000 // 64,
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=25,
    verbose=1,callbacks=[checkpoint,clr_triangular]
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25
  1/625 [..............................] - ETA: 30:17 - loss: 1.0681 - gender_output_loss: 1.2200e-04 - image_quality_output_loss: 0.0859 - age_output_loss: 0.1154 - weight_output_loss: 0.3655 - bag_output_loss: 0.0541 - footwear_output_loss: 0.0607 - pose_output_loss: 0.0081 - emotion_output_loss: 0.3782 - gender_output_acc: 1.0000 - image_quality_output_acc: 1.0000 - age_output_acc: 0.9688 - weight_output_acc: 0.9375 - bag_output_acc: 1.0000 - footwear_output_acc: 0.9375 - pose_output_acc: 1.0000 - emotion_output_acc: 0.8750

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (4.453933). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


  2/625 [..............................] - ETA: 39:24 - loss: 1.4933 - gender_output_loss: 0.0038 - image_quality_output_loss: 0.1033 - age_output_loss: 0.2159 - weight_output_loss: 0.4365 - bag_output_loss: 0.1329 - footwear_output_loss: 0.1116 - pose_output_loss: 0.0295 - emotion_output_loss: 0.4598 - gender_output_acc: 1.0000 - image_quality_output_acc: 0.9844 - age_output_acc: 0.9375 - weight_output_acc: 0.8906 - bag_output_acc: 0.9375 - footwear_output_acc: 0.9531 - pose_output_acc: 1.0000 - emotion_output_acc: 0.8281    

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (2.231895). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


625/625 [==============================] - 151s 241ms/step - loss: 1.1344 - gender_output_loss: 0.0107 - image_quality_output_loss: 0.0811 - age_output_loss: 0.1749 - weight_output_loss: 0.2635 - bag_output_loss: 0.1087 - footwear_output_loss: 0.0648 - pose_output_loss: 0.0183 - emotion_output_loss: 0.4125 - gender_output_acc: 0.9961 - image_quality_output_acc: 0.9754 - age_output_acc: 0.9469 - weight_output_acc: 0.9114 - bag_output_acc: 0.9609 - footwear_output_acc: 0.9778 - pose_output_acc: 0.9931 - emotion_output_acc: 0.8466 - val_loss: 0.7625 - val_gender_output_loss: 9.2217e-04 - val_image_quality_output_loss: 0.0315 - val_age_output_loss: 0.0862 - val_weight_output_loss: 0.1944 - val_bag_output_loss: 0.0477 - val_footwear_output_loss: 0.0281 - val_pose_output_loss: 0.0018 - val_emotion_output_loss: 0.3719 - val_gender_output_acc: 1.0000 - val_image_quality_output_acc: 0.9950 - val_age_output_acc: 0.9899 - val_weight_output_acc: 0.9521 - val_bag_output_acc: 0.9929 - val_footwear_o